In [1]:
import datetime as dt
import polars as pl

from factor_momentum import RollingPCA, FactorMomentumSignal, FACTORS
from sf_quant.data import load_factors


/home/boobus/projects/factor_momentum/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-11-10 17:34:09,089	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
start = dt.date(2001,1,1); end = dt.date(2020,1,1); factor_returns = load_factors(start,end, FACTORS).lazy()
pca_engine = RollingPCA(n_components=5, lookback_window=30)

In [3]:
pca_engine.fit_transform_rolling_monthly(start, end, factor_returns)

Fitting rolling PCA...


Rolling PCA: 100%|██████████| 227/227 [00:00<00:00, 527.01it/s]


Transforming rolling PCA...


Transforming PCA: 100%|██████████| 227/227 [00:00<00:00, 553.21it/s]


date,pc0,pc1,pc2,pc3,pc4
date,f64,f64,f64,f64,f64
2001-02-01,-1.464667,-1.143598,0.180363,0.551172,1.143439
2001-02-01,-1.481167,-0.150589,0.239467,0.383877,-0.276733
2001-02-01,-0.997846,-0.57137,-0.960766,0.96356,0.80466
2001-02-01,0.441192,-0.101307,-0.564052,-1.315537,-1.572022
2001-02-01,0.071369,0.954979,-0.923436,1.117135,-1.701075
…,…,…,…,…,…
2019-12-02,-0.815778,-0.496197,0.021434,-0.190844,-0.694501
2019-12-02,-0.641084,2.216797,0.654364,0.226481,-0.006345
2019-12-02,-1.291248,1.671811,-1.161495,-0.417639,0.653435
